In [1]:
import os,sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.load_data import *
from scripts.data_preparation import *
from scripts.training_and_evaluation import *

In [3]:
fra_data_path = '../../Data/Fraud_Data.csv'
# IpAd_data_path = '../../Data/IpAddress_to_Country.csv'
cred_data_path = '../../Data/creditcard.csv/creditcard.csv'

loaded_data_fra = load_data(fra_data_path)
loaded_data_cred = load_data(cred_data_path)

loaded_data_fra.head(5)
# loaded_data_cred.head(5)


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [4]:
# Prepare data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = data_preparation(loaded_data_fra)
X_train_cc, X_test_cc, y_train_cc, y_test_cc = data_preparation(loaded_data_cred)

In [5]:
# Define models
models = define_models()

In [6]:
print(X_test_fraud)
print(X_test_cc)

        user_id  purchase_value  age    ip_address  signup_year  signup_month  \
92865    324640              45   33  3.315147e+09         2015             1   
50746    244111              54   36  9.916283e+08         2015             6   
128421    81345              62   19  2.744427e+08         2015             4   
146243    25395              66   49  2.654816e+09         2015             8   
44524     11703              34   45  1.419980e+09         2015             2   
...         ...             ...  ...           ...          ...           ...   
41601    224949              29   31  3.269694e+09         2015             7   
52668    395586              29   44  2.294934e+08         2015             2   
69482    149769               9   26  1.846086e+09         2015             2   
144858   287465              25   41  3.139010e+09         2015             5   
70085     66029              48   36  2.778374e+08         2015             5   

        signup_day  signup_

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import SparsePCA  # Use SparsePCA for better memory handling
from scipy.sparse import csr_matrix
import numpy as np

# Reduce dataset size for experimentation (use a very small subset)
X_train_fraud_sampled, _, y_train_fraud_sampled, _ = train_test_split(
    X_train_fraud, y_train_fraud, train_size=0.005, random_state=42  # Use 0.5% of the data
)

# Reduce dimensionality using Sparse PCA
sparse_pca = SparsePCA(n_components=50)  # Retain 50 principal components
X_train_fraud_sampled_pca = sparse_pca.fit_transform(X_train_fraud_sampled)

# Transform the test data
X_test_fraud_pca = sparse_pca.transform(X_test_fraud)

# Convert to sparse matrices
X_train_fraud_sampled_sparse = csr_matrix(X_train_fraud_sampled_pca)
X_test_fraud_sparse = csr_matrix(X_test_fraud_pca)

# Evaluate models for Fraud Data
fraud_results = evaluate_models(X_train_fraud_sampled_sparse, y_train_fraud_sampled, X_test_fraud_sparse, y_test_fraud, models)
for model_name, metrics in fraud_results.items():
    print(f"Model: {model_name}\n{metrics['report']}\nConfusion Matrix:\n{metrics['confusion_matrix']}\nAccuracy: {metrics['accuracy']:.4f}\n")

In [9]:
import psutil
import os

process = psutil.Process(os.getpid())
print(f"Memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")

Memory usage: 11231.14 MB


In [ ]:
# Evaluate models for Fraud Data
fraud_results = evaluate_models(X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud, models)
for model_name, metrics in fraud_results.items():
    print(f"Model: {model_name}\n{metrics['report']}\nConfusion Matrix:\n{metrics['confusion_matrix']}\nAccuracy: {metrics['accuracy']:.4f}\n")

In [ ]:
# Log results
log_results(models, X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud)